# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Model training & UI Exploration</span>

<span style="font-width:bold; font-size: 1.4rem;">In this last notebook, we will train a model on the dataset we created in the previous tutorial. We will train our model using standard Python and Scikit-learn, although it could just as well be trained with other machine learning frameworks such as PySpark, TensorFlow, and PyTorch. We will also show some of the exploration that can be done in Hopsworks, notably the search functions and the lineage. </span>

## **🗒️ This notebook is divided in 3 main sections:** 
1. **Loading the training data**
2. **Train the model**
3. **Register and deploy model** 

![tutorial-flow](images/03_model.png)

In [1]:
from ast import literal_eval
import pandas as pd

import hsfs

## Connect to hsfs and retrieve datasets for training and evaluation 

In [2]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


### get feature view objects 

In [3]:
feature_view = fs.get_feature_view('aml_feature_view', 1)

## <span style="color:#ff5f27;"> 🪝 Training Dataset retreival </span>

To retrieve training data from storage (already materialised) or from feature groups direcly we can use `get_training_dataset_splits` or `get_training_dataset` methods. If version is not provided or provided version has not already existed, it creates a new version of training data according to given arguments and returns a dataframe. If version is provided and has already existed, it reads training data from storage or feature groups and returns a dataframe. If split is provided, it reads the specific split.

In [4]:
train_x, train_y  = feature_view.get_training_data(1)

2022-06-19 16:10:10,615 INFO: USE `aml_demo_featurestore`
2022-06-19 16:10:11,430 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg2`.`type` `type`, `fg2`.`is_sar` `is_sar`, `fg2`.`id` `join_pk_id`, `fg2`.`tran_timestamp` `join_evt_tran_timestamp`, `fg0`.`monthly_in_count` `monthly_in_count`, `fg0`.`monthly_in_total_amount` `monthly_in_total_amount`, `fg0`.`monthly_in_mean_amount` `monthly_in_mean_amount`, `fg0`.`monthly_in_std_amount` `monthly_in_std_amount`, `fg0`.`monthly_out_count` `monthly_out_count`, `fg0`.`monthly_out_total_amount` `monthly_out_total_amount`, `fg0`.`monthly_out_mean_amount` `monthly_out_mean_amount`, `fg0`.`monthly_out_std_amount` `monthly_out_std_amount`, RANK() OVER (PARTITION BY `fg2`.`id`, `fg2`.`tran_timestamp` ORDER BY `fg0`.`tran_timestamp` DESC) pit_rank_hopsworks
FROM `aml_demo_featurestore`.`party_fg_1` `fg2`
INNER JOIN `aml_demo_featurestore`.`transactions_monthly_fg_1` `fg0` ON `fg2`.`id` = `fg0`.`id` AND `fg2`.`tran_timestamp` >= `fg0`.`tran_timest

In [5]:
# explode embeddings
train_x['graph_embeddings'] = train_x['graph_embeddings'].apply(literal_eval)
train_x = train_x.merge( pd.DataFrame(train_x['graph_embeddings'].to_list()).add_prefix('emb_'), left_index=True, right_index=True).drop('graph_embeddings', axis = 1)

We are going to train [gan for anomaly detection](https://arxiv.org/pdf/1905.11034.pdf). Durring training step  we will provide only features of accounts that have never been reported for suspicios activity.  We will disclose previously reported accounts to the model only in evaluation step.   

In [6]:
non_sar_transactions = train_x[train_y.values == 0]

In [7]:
non_sar_transactions = non_sar_transactions.dropna()

##### Now lets define Tensorflow Dataset as we are going to train keras tensorflow model

In [8]:
import tensorflow as tf

def windowed_dataset(dataset, window_size, batch_size):
    ds = dataset.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x.batch(window_size))
    return ds.batch(batch_size,True).prefetch(1)

training_dataset = tf.data.Dataset.from_tensor_slices(tf.cast(non_sar_transactions.values, tf.float32))
training_dataset = windowed_dataset(training_dataset, window_size=2, batch_size=16)
training_dataset

<PrefetchDataset shapes: (16, None, 41), types: tf.float32>

## <span style="color:#ff5f27;"> 🏃 Train Model</span>

Next we'll train a model. Here, we set the class weight of the positive class to be twice as big as the negative class.

## Model architecture
![tutorial-flow](images/model_architecture.png)

In [9]:
class GanEncAnomalyDetector(tf.keras.Model):
    
    def __init__(self, input_dim):
        super(GanEncAnomalyDetector, self).__init__()
        
        self.input_dim = input_dim
        self.latent_dim = [1, input_dim[1]] 
        self.d_steps = 3
        self.gp_weight = 10 
        
        self.encoder = self.make_encoder_model(self.input_dim)
        self.generator = self.make_generator(self.input_dim, self.latent_dim)
        self.discriminator = self.make_discriminator_model(self.input_dim)

        self.mse = tf.keras.losses.MeanSquaredError()
        
        self.epoch_e_loss_avg = tf.keras.metrics.Mean(name="epoch_e_loss_avg")
        self.epoch_d_loss_avg = tf.keras.metrics.Mean(name="epoch_d_loss_avg")
        self.epoch_g_loss_avg = tf.keras.metrics.Mean(name="epoch_g_loss_avg")
        self.epoch_a_score_avg = tf.keras.metrics.Mean(name="epoch_a_score_avg")

        @property
        def metrics(self):
            return [
                self.epoch_e_loss_avg,
                self.epoch_d_loss_avg,
                self.epoch_g_loss_avg,
                self.epoch_a_score_avg,
            ]

    # define model architectures
    def make_encoder_model(self, input_dim):
        inputs = tf.keras.layers.Input(shape=(input_dim[0],input_dim[1]))
        x = tf.keras.layers.Conv1D(filters = 64, kernel_size= 1,padding='same', kernel_initializer="uniform")(inputs)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)    
        x = tf.keras.layers.MaxPooling1D(pool_size=2, padding='same')(x)
        x = tf.keras.layers.Conv1D(filters = input_dim[1], kernel_size= 1,padding='same',  kernel_initializer="uniform")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)    
        x = tf.keras.layers.MaxPooling1D(pool_size=2, padding='same')(x)    
        encoder = tf.keras.Model(inputs=inputs, outputs=x, name="encoder_model")
        return encoder

    def make_generator(self, input_dim, latent_dim):
        latent_inputs = tf.keras.layers.Input(shape=(latent_dim[0],latent_dim[1]))
        x = tf.keras.layers.Conv1D(filters = 8, kernel_size= 1,padding='same', kernel_initializer="uniform")(latent_inputs) 
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)    
        x = tf.keras.layers.UpSampling1D(2)(x) 
        x = tf.keras.layers.Conv1D(filters = 16, kernel_size= 1,padding='same', kernel_initializer="uniform")(x) 
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)    
        #x = tf.keras.layers.UpSampling1D(2)(x) 
        x = tf.keras.layers.Conv1D(filters = input_dim[1], kernel_size= 1,padding='same', kernel_initializer="uniform")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)    
        generator = tf.keras.Model(inputs=latent_inputs, outputs=x, name="generator_model")        
        return generator

    def make_discriminator_model(self, input_dim):
        inputs = tf.keras.layers.Input(shape=(input_dim[0],input_dim[1]))
        x = tf.keras.layers.Conv1D(filters = 128, kernel_size= 1,padding='same', kernel_initializer="uniform")(inputs)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)    
        x = tf.keras.layers.MaxPooling1D(pool_size=2, padding='same')(x)
        x = tf.keras.layers.Conv1D(filters = 64, kernel_size= 1,padding='same', kernel_initializer="uniform")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)    

        # dense output layer
        x = tf.keras.layers.Flatten()(x)    
        x = tf.keras.layers.LeakyReLU(0.2)(x)
        x = tf.keras.layers.Dense(128)(x)
        x = tf.keras.layers.LeakyReLU(0.2)(x)
        prediction = tf.keras.layers.Dense(1)(x)
        discriminator = tf.keras.Model(inputs=inputs, outputs=prediction, name="discriminator_model" )               
        return discriminator
        
    # Training function
    @tf.function
    def train_step(self, real_data):
        if isinstance(real_data, tuple):
            real_data = real_data[0]

        # Get the batch size
        batch_size = tf.shape(real_data)[0]

        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 1a. Train the encoder and get the encoder loss
        # 2. Train the discriminator and get the discriminator loss
        # 3. Calculate the gradient penalty
        # 4. Multiply this gradient penalty with a constant weight factor
        # 5. Add the gradient penalty to the discriminator loss
        # 6. Return the generator and discriminator losses as a loss dictionary

        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator. Here we will train it for 3 extra steps
        # as compared to 5 to reduce the training time.
        for i in range(self.d_steps):
            # Get the latent vector
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim[0], self.latent_dim[1])), 
            with tf.GradientTape() as tape:
                # Generate fake data from the latent vector
                fake_data = self.generator(random_latent_vectors, training=True)

                #(somewhere here step forward?)
                # Get the logits for the fake data
                fake_logits = self.discriminator(fake_data, training=True)
                # Get the logits for the real data
                real_logits = self.discriminator(real_data, training=True)

                # Calculate the discriminator loss using the fake and real sample logits
                d_cost = self.discriminator_loss(real_sample=real_logits, fake_sample=fake_logits)
                # Calculate the gradient penalty
                gp = self.gradient_penalty(real_data, fake_data)
                # Add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * self.gp_weight

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            self.d_optimizer.apply_gradients(
                zip(d_gradient, self.discriminator.trainable_variables)
            )

        # Train the generator
        # Get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim[0], self.latent_dim[1]))
        with tf.GradientTape() as tape:
            # Generate fake data using the generator
            generated_data = self.generator(random_latent_vectors, training=True)
            # Get the discriminator logits for fake data
            gen_sample_logits = self.discriminator(generated_data, training=True)
            # Calculate the generator loss
            g_loss = self.generator_loss(gen_sample_logits)

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )

        # Train the encoder
        with tf.GradientTape() as tape:
            generated_data = self.generator(random_latent_vectors, training=True)
            # Compress generate fake data from the latent vector
            encoded_fake_data = self.encoder(generated_data, training=True)
            # Reconstruct encoded generate fake data
            generator_reconstructed_encoded_fake_data = self.generator(encoded_fake_data, training=True)
            # Encode the latent vector
            encoded_random_latent_vectors = self.encoder(tf.random.normal(shape=(batch_size, self.input_dim[0], self.input_dim[1])), 
                                                         training=True)
            # Calculate encoder loss
            e_loss = self.encoder_loss(generated_data, generator_reconstructed_encoded_fake_data)

        # Get the gradients w.r.t the generator loss
        enc_gradient = tape.gradient(e_loss, self.encoder.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.e_optimizer.apply_gradients(
            zip(enc_gradient, self.encoder.trainable_variables)
        )

        anomaly_score = self.compute_anomaly_score(real_data)

        self.epoch_d_loss_avg.update_state(d_loss)
        self.epoch_g_loss_avg.update_state(g_loss)
        self.epoch_e_loss_avg.update_state(e_loss)
        self.epoch_a_score_avg.update_state(anomaly_score["anomaly_score"])

        return {"d_loss": d_loss, "g_loss": g_loss, "e_loss": e_loss, "anomaly_score": anomaly_score["anomaly_score"]}

    @tf.function
    def test_step(self, input):
        if isinstance(input, tuple):
            input = input[0]
        
        batch_size = tf.shape(input)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim[0], self.latent_dim[1]))
        # Generate fake data using the generator
        generated_data = self.generator(random_latent_vectors, training=False)
        # Get the discriminator logits for fake data
        gen_sample_logits = self.discriminator(generated_data, training=False)
        # Calculate the generator loss
        g_loss = self.generator_loss(gen_sample_logits)

        
        # Compress generate fake data from the latent vector
        encoded_fake_data = self.encoder(generated_data, training=False)
        # Reconstruct encoded generate fake data
        generator_reconstructed_encoded_fake_data = self.generator(encoded_fake_data, training=False)

        # Calculate encoder loss
        e_loss = self.encoder_loss(generated_data, generator_reconstructed_encoded_fake_data)
        
        anomaly_score = self.compute_anomaly_score(input)
        return {
            "g_loss": g_loss,
            "e_loss": e_loss,
            "anomaly_score": anomaly_score["anomaly_score"]
        }
    
    # define custom server function
    @tf.function
    def serve_function(self, input):
        return self.compute_anomaly_score(input)

    def call(self, input):
        if isinstance(input, tuple):
            input = input[0]
        
        encoded = self.encoder(input)
        decoded = self.generator(encoded)
        anomaly_score = self.compute_anomaly_score(input)
        return anomaly_score["anomaly_score"], decoded

    def compile(self):
        super(GanEncAnomalyDetector, self).compile()     
        # Define optimizers
        self.e_optimizer = tf.keras.optimizers.SGD(lr=0.00001, clipnorm=0.01)        
        self.d_optimizer = tf.keras.optimizers.SGD(lr=0.00001, clipnorm=0.01)
        self.g_optimizer = tf.keras.optimizers.SGD(lr=0.00001, clipnorm=0.01)

    def gradient_penalty(self, real_data, fake_data):
        """ Calculates the gradient penalty.
        This loss is calculated on an interpolated sample
        and added to the discriminator loss.
        """
        # Get the interpolated sample
        real_data_shape = tf.shape(real_data)
        alpha = tf.random.normal(shape=[real_data_shape[0], real_data_shape[1], real_data_shape[2]], mean=0.0, stddev=2.0, dtype=tf.dtypes.float32)
        #alpha = tf.random_uniform([self.batch_size, 1], minval=-2, maxval=2, dtype=tf.dtypes.float32)
        interpolated = (alpha * real_data) + ((1 - alpha) * fake_data)

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated sample.
            pred = self.discriminator(interpolated, training=True)

        # 2. Calculate the gradients w.r.t to this interpolated sample.
        grads = gp_tape.gradient(pred, [interpolated])[0]
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[-2, -1]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp    
        
    def encoder_loss(self,generated_fake_data, generator_reconstructed_encoded_fake_data):
        generator_reconstracted_data = tf.cast(generator_reconstructed_encoded_fake_data, tf.float32)
        loss = self.mse(generated_fake_data, generator_reconstracted_data)
        beta_cycle_gen = 10.0
        loss = loss * beta_cycle_gen
        return loss

    # Define the loss functions for the discriminator,
    # which should be (fake_loss - real_loss).
    # We will add the gradient penalty later to this loss function.
    def discriminator_loss(self, real_sample, fake_sample):
        real_loss = tf.reduce_mean(real_sample)
        fake_loss = tf.reduce_mean(fake_sample)
        return fake_loss - real_loss

    # Define the loss functions for the generator.
    def generator_loss(self, fake_sample):
        return -tf.reduce_mean(fake_sample)
    
    def compute_anomaly_score(self, input):
        """anomaly score.
          See https://arxiv.org/pdf/1905.11034.pdf for more details
        """
        # Encode the real data
        encoded_real_data = self.encoder(input, training=False)
        # Reconstruct encoded real data
        generator_reconstructed_encoded_real_data = self.generator(encoded_real_data, training=False)
        # Calculate distance between real and reconstructed data (Here may be step forward?)
        gen_rec_loss_predict = self.mse(input,generator_reconstructed_encoded_real_data)

        # # Compute anomaly score
        # real_to_orig_dist_predict = tf.math.reduce_sum(tf.math.pow(encoded_random_latent - encoded_real_data, 2), axis=[-1])
        # anomaly_score = (gen_rec_loss_predict * self.anomaly_alpha) + ((1 - self.anomaly_alpha) * real_to_orig_dist_predict)
        anomaly_score = gen_rec_loss_predict
        return {'anomaly_score': anomaly_score} 
    

In [10]:
model = GanEncAnomalyDetector([2, 41])
model.compile()

In [11]:
for layer in model.layers:
    print(layer.name, layer.output_shape)

encoder_model (None, 1, 41)
generator_model (None, 2, 41)
discriminator_model (None, 1)


In [12]:
history = model.fit(training_dataset, epochs=2, verbose=0)

In [13]:
metrics={'loss': history.history["g_loss"][0]} 

## <span style="color:#ff5f27;">  Register anomaly detection model to hopsworks model registry </span>

In [14]:
import os
import uuid

# hops model registry library
import hsml
    
# save to the model registry
export_path = os.getcwd() + '/model-' + str(uuid.uuid4())
print('Exporting trained model to: {}'.format(export_path))
    
call = model.serve_function.get_concrete_function(tf.TensorSpec([None,None,None], tf.float32))
tf.saved_model.save(model, export_path, signatures=call)

conn_hsml = hsml.connection()
mr = conn_hsml.get_model_registry()

# define sample input 
test_dataset = tf.data.Dataset.from_tensor_slices(tf.cast(non_sar_transactions.values, tf.float32))
test_dataset = windowed_dataset(test_dataset, window_size=2, batch_size=1)
input_example = next(iter(test_dataset)).numpy()

mr_model = mr.tensorflow.create_model(
    name="aml_model",
    metrics=metrics,
    description="Adversarial anomaly detection model.",
    input_example=input_example
)

mr_model.save(export_path)

Exporting trained model to: /srv/hops/staging/private_dirs/8deec0164f68faee054f68bf17951435dcd209946539a0f4f9cd0e48f91fd3d7/model-0f743f48-5af4-48c1-a7b5-d4b9babc78a5
2022-06-19 16:12:35,934 WARNING: Skipping full serialization of Keras layer <__main__.GanEncAnomalyDetector object at 0x7f41db9c95e0>, because it is not built.
2022-06-19 16:12:51,427 WARNING: Found untraced functions such as test_step while saving (showing 1 of 1). These functions will not be directly callable after loading.
2022-06-19 16:12:51,773 WARNING: Found untraced functions such as test_step while saving (showing 1 of 1). These functions will not be directly callable after loading.
2022-06-19 16:12:53,962 INFO: Assets written to: /srv/hops/staging/private_dirs/8deec0164f68faee054f68bf17951435dcd209946539a0f4f9cd0e48f91fd3d7/model-0f743f48-5af4-48c1-a7b5-d4b9babc78a5/assets
Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://hopsworks0.logicalclocks.com/p/119/models/aml_model/1


Model(name: 'aml_model', version: 1)

## <span style="color:#ff5f27;"> Deploy model </span>

## About Model Serving
Models can be served via KFServing or "default" serving, which means a Docker container exposing a Flask server. For KFServing models, or models written in Tensorflow, you do not need to write a prediction file (see the section below). However, for sklearn models using default serving, you do need to proceed to write a prediction file.

In order to use KFServing, you must have Kubernetes installed and enabled on your cluster.



In [16]:
# Use the model name from the previous notebook.
model = mr.get_model("aml_model", version=1)

# Give it any name you want
deployment = model.deploy(
    name="amldeployment",
    model_server="TENSORFLOW_SERVING", 
    serving_tool="KSERVE",
)

In [19]:
print("Deployment: " + deployment.name)
deployment.describe()

Deployment: amldeployment
{
    "artifact_version": 0,
    "batching_configuration": {
        "batching_enabled": false
    },
    "created": "2022-06-19T16:15:14.24Z",
    "creator": "Admin Admin",
    "id": 2,
    "inference_logging": "NONE",
    "model_name": "aml_model",
    "model_path": "/Projects/aml_demo/Models/aml_model",
    "model_server": "TENSORFLOW_SERVING",
    "model_version": 1,
    "name": "amldeployment",
    "predictor": null,
    "predictor_resources": {
        "limits": {
            "cores": 1,
            "gpus": 0,
            "memory": 1024
        },
        "requests": {
            "cores": 1,
            "gpus": 0,
            "memory": 1024
        }
    },
    "requested_instances": 1,
    "serving_tool": "KSERVE"
}


The deployment has now been registered. However, to start it you need to run:

In [20]:
deployment.start()

  0%|          | 0/1 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


### For trouble shooting one can use `get_logs` method

In [21]:
deployment.get_logs()

Explore all the logs and filters in the Kibana logs at https://hopsworks0.logicalclocks.com/p/119/deployments/2

Instance name: amldeployment-predictor-default-00001-deployment-55cfb55d5rj8zb
2022-06-19 16:15:41.935199: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:206] Restoring SavedModel bundle.
2022-06-19 16:15:41.969706: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199985000 Hz
2022-06-19 16:15:42.183170: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /mnt/models/0
2022-06-19 16:15:42.337756: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 782650 microseconds.
2022-06-19 16:15:42.377744: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /mnt/models/0/assets.extra/tf_serving_warmup_requests
2022-06-19 16:15:

### Using the deployment
Let's use the input example that we registered together with the model to query the deployment.

In [25]:
data = {
    "inputs": model.input_example
}

deployment.predict(data)

{'outputs': 0.75004369}

In [23]:
# For trouble shooting one can use get_logs methods
deployment.get_logs()

Explore all the logs and filters in the Kibana logs at https://hopsworks0.logicalclocks.com/p/119/deployments/2

Instance name: amldeployment-predictor-default-00001-deployment-55cfb55d5rj8zb
2022-06-19 16:15:41.969706: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199985000 Hz
2022-06-19 16:15:42.183170: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /mnt/models/0
2022-06-19 16:15:42.337756: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 782650 microseconds.
2022-06-19 16:15:42.377744: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /mnt/models/0/assets.extra/tf_serving_warmup_requests
2022-06-19 16:15:42.381680: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: amldeployment version

## Use REST endpoint
You can also use a REST endpoint for your model. To do this you need to create an API key with 'serving' enabled, and retrieve the endpoint URL from the Model Serving UI.

Go to the Model Serving UI and click on the eye icon next to a model to retrieve the endpoint URL. The shorter URL is an internal endpoint that you can only reach from within Hopsworks. If you want to call it from outside, you need one of the longer URLs. 

![serving-endpoints](images/serving_endpoints.gif)

In [ ]:
import os
import requests

mr = project.get_model_registry()

# Use the model name from the previous notebook.
model = mr.get_model("fraud_tutorial_model", version=1)

test_inputs = [model.input_example]

API_KEY = "..."  # Put your API key here.
MODEL_SERVING_URL = "..." # Put model serving endppoint here.
HOST_NAME = "..." # Put your hopsworks model serving hostname here 

data = {"inputs": test_inputs}
headers = {
    "Content-Type": "application/json", "Accept": "application/json",
    "Authorization": f"ApiKey {API_KEY}",
    "Host": HOST_NAME}

response = requests.post(MODEL_SERVING_URL, verify=False, headers=headers, json=data)
response.json()

In [ ]:
# Now lets test feature vectors from online store
ids_to_score = ["0016359b", 
                "001dcc27", 
                "0054a022", 
                "00d6b609", 
                "00e14860", 
                "00e39a1b", 
                "014ed5cb", 
                "01ce3306", 
                "01fa19ae", 
                "01fa1d01", 
                "036dce03", 
                "03e09be4", 
                "04b23f4b"]

def flat2gen(alist):
    for item in alist:
        if isinstance(item, list):
            for subitem in item: yield subitem
        else:
            yield item

def model_server(input):
    data = {"inputs": [input]}
    return deployment.predict(data)

for node_id in ids_to_score:
    serving_vector = np.array(list(flat2gen(feature_view.get_feature_vector({'id': node_id})))).reshape(1,41).tolist()
    print(" anomaly score for node_id ", node_id, " : ",   model_server(serving_vector)["outputs"])


## Stop Deployment
To stop the deployment we simply run:

In [ ]:
deployment.stop()

## <span style="color:#ff5f27;"> 🎁 Wrapping things up </span>

In this module we perforemed feature engineering, created feature view and traning dataset, trained advance anomaly detection model and depoyed it in production. You can try this out on hopsworks.ai and contuct us for any futher details.

<img src="images/contuct_us.png" width="400px"></img>